In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## 'r' -> to read
f = open('Minerskape.txt', 'r', encoding = 'utf-8')

In [ ]:
data = f.read()

In [ ]:
print(data)

In [ ]:
# Refer regex101 for using regular expression
import re

In [ ]:
pattern = '\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
# To have data splitted according to dates from text
# [1:] to remove first empty element
message = re.split(pattern, data)[1:]
len(message)

In [ ]:
print(message)

In [ ]:
# To filter dates from text
dates = re.findall(pattern, data)
len(dates)

In [ ]:
print(dates)

In [ ]:
df = pd.DataFrame({'user_message':message,'date':dates})
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%y, %H:%M - ')
df.head(10)

In [ ]:
# Identifying the user
user = []
messages = []
for chat in df['user_message']:
    if ':' in chat:
        entry = re.split('([\w\W]+?):\s',chat)
        user.append(entry[1])
        messages.append(entry[2])
    else:
        user.append('group notification')
        messages.append(chat)
df['user_message'] = messages
df['user'] = user

In [ ]:
df

## Identifying Year, Month, Day, Hour and Minute of message

In [ ]:
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df

In [ ]:
df.shape

In [ ]:
# Number of messages sent by 'Mayank Cu'
df[df['user'] == 'Mayank Cu'].shape[0]

In [ ]:
# Storing the messages word by word in a list
# message.split() will give a list
# append will exactly copy the given list but extend will copy the elements of list
# extend and append are same when we do not pass a collection
words = []
for message in df['user_message']:
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
words

In [ ]:
## Identifying the media messages
df[df['user_message'] == "<Media omitted>\n"].shape[0]

# !pip install urlextract

In [ ]:
from urlextract import URLExtract
extractor = URLExtract()

In [ ]:
## Identifying the links
links =[]
for message in df['user_message']:
    links.extend(extractor.find_urls(message))
len(links)

In [ ]:
## Finding and plotting count of each user
x = df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name = x.index
count = x.values

In [ ]:
plt.bar(name, count)
plt.xticks(rotation = 'vertical')

In [ ]:
df['user'].value_counts()

In [ ]:
## Finding the most active user via percentage of messages sent
round((df['user'].value_counts()/df.shape[0]) * 100,2).reset_index().rename(columns = {'user': 'Name', 'count': 'Percentage'})

In [ ]:
## Finding the most frequent words
from collections import Counter
pd.DataFrame(Counter(words).most_common(20)).rename(columns = {0: 'Word', 1: 'Count'})

## Identifying messages with no user, media omitted and deleted message

In [ ]:
df1= df[df['user']!='group notification']

In [ ]:
df2=df1[df1['user_message']!= '<Media omitted>\n']

In [ ]:
temp = df2[df2['user_message'] != 'This message was deleted\n']

In [ ]:
temp

## Removing stopwords from user messages and identifying admin messages

In [ ]:
f = open('stop_hinglish.txt','r')
stop_words = f.read()
print(stop_words)

In [ ]:
len(stop_words)

In [ ]:
words = []
for message in temp['user_message']:
    for word in message.lower().split():
        if word not in stop_words:
            if word == '@917506941616':
                words.append('admin tagged')
            else:
                words.append(word)

In [ ]:
words

In [ ]:
len(words)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(20))

## Finding emojis in user messages

# !pip install emoji

In [ ]:
import emoji

In [ ]:
## emoji's code -> emoji's unicode data -> number for machine and symbol for human
emojis = []
for i in df['user_message']:
    emojis.extend([c for c in i if c in emoji.EMOJI_DATA])

In [ ]:
emojis

In [ ]:
pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis)))).rename(columns = {0: 'Emoji', 1: 'Count'})

In [ ]:
df

In [ ]:
# Adding month number
df['month_num'] = df['date'].dt.month

In [ ]:
df

## Plotting timeline for user messages

In [ ]:
timeline = df.groupby(['year','month','month_num']).count()['user_message'].reset_index()

In [ ]:
timeline

In [ ]:
time =[]
for i in range(timeline.shape[0]):
    time.append(timeline['month'][i] + " - " + str(timeline['year'][i]))

In [ ]:
time

In [ ]:
timeline['time'] = time

In [ ]:
timeline

In [ ]:
# group based on year and then sort based on month number
timeline1 = timeline[timeline['year']==2021].sort_values('month_num').reset_index(drop=True)
timeline2 = timeline[timeline['year']==2022].sort_values('month_num').reset_index(drop=True)
timeline = pd.concat([timeline1,timeline2],axis=0)
timeline

In [ ]:
plt.plot(timeline['time'], timeline['user_message'])
plt.xticks(rotation='vertical')
plt.show()

## Filtering user messages by date only

In [ ]:
df['only_date'] = df['date'].dt.date

In [ ]:
df

In [ ]:
timeline_date = df.groupby('only_date').count()['user_message'].reset_index()

In [ ]:
timeline_date